In [1]:
import tensorflow as tf
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib.image import imread
from PIL import Image
from random import shuffle
%matplotlib inline

In [18]:
cat = [1, 0]
dog = [0, 1]
SIZE = 50

keepRate = 0.8
LR = 0.05
epochs = 20
batchSize = 10

TRAIN_DIR = os.path.join(os.getcwd(), 'train')
TEST_DIR = os.path.join(os.getcwd(), 'test')

In [19]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat': return cat
    elif word_label == 'dog': return dog
    
def resize_data():
    for img in os.listdir(TRAIN_DIR):
        path = os.path.join(TRAIN_DIR, img)
        img = Image.open(path)
        img = img.resize((SIZE, SIZE), PIL.Image.ANTIALIAS)
        img.save(path)
        
    for img in os.listdir(TEST_DIR):
        path = os.path.join(TEST_DIR, img)
        img = Image.open(path)
        img = img.resize((SIZE, SIZE), PIL.Image.ANTIALIAS)
        img.save(path)

def prep_data():
    train_data_imgs = []
    train_data_lbls = []
    
    for img in os.listdir(TRAIN_DIR):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        img = imread(path)
        train_data_imgs.append(np.array(img))
        train_data_lbls.append(np.array(label))
        
    test_data = []
    for img in os.listdir(TEST_DIR):
        path = os.path.join(TEST_DIR, img)
        img = imread(path)
        test_data.append(img)
        
    return train_data_imgs, train_data_lbls, test_data

In [20]:
# resize_data()

In [21]:
# train_data_imgs, train_data_lbls, test_data = prep_data()

In [22]:
# print(np.array(train_data_imgs).shape)
# print(np.array(train_data_lbls).shape)
# print(np.array(test_data).shape)

In [23]:
# np.save('train_data_imgs.npy', train_data_imgs)
# np.save('train_data_lbls', train_data_lbls)
# np.save('test_data.npy', test_data)

train_data_imgs = np.load('train_data_imgs.npy')
train_data_lbls = np.load('train_data_lbls.npy')
test_data = np.load('test_data.npy')

In [24]:
print(np.array(train_data_imgs).shape)
print(np.array(train_data_lbls).shape)
print(np.array(test_data).shape)

(25000, 50, 50, 3)
(25000, 2)
(12500, 50, 50, 3)


In [25]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [26]:
x = tf.placeholder(tf.float32, shape=[None, SIZE, SIZE, 3])
y = tf.placeholder(tf.float32, shape=[None, 2])

weights = {
    'Wconv1': tf.Variable(tf.random_normal([5, 5, 3, 32])),
    'Wconv2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'Wconv3': tf.Variable(tf.random_normal([5, 5, 64, 128])),
    'Wconv4': tf.Variable(tf.random_normal([5, 5, 128, 64])),
    'Wconv5': tf.Variable(tf.random_normal([5, 5, 64, 32])),
    'Wfc': tf.Variable(tf.random_normal([2*2*32, 1024])),
    'Wout': tf.Variable(tf.random_normal([1024, 2]))
}

biases = {
    'bconv1': tf.Variable(tf.zeros([32])),
    'bconv2': tf.Variable(tf.zeros([64])),
    'bconv3': tf.Variable(tf.zeros([128])),
    'bconv4': tf.Variable(tf.zeros([64])),
    'bconv5': tf.Variable(tf.zeros([32])),
    'bfc': tf.Variable(tf.zeros([1024])),
    'bout': tf.Variable(tf.zeros([2]))
}

convnet = tf.nn.relu(conv2d(x, weights['Wconv1']) + biases['bconv1'])
convnet = maxpool2d(convnet)

convnet = tf.nn.relu(conv2d(convnet, weights['Wconv2']) + biases['bconv2'])
convnet = maxpool2d(convnet)

convnet = tf.nn.relu(conv2d(convnet, weights['Wconv3']) + biases['bconv3'])
convnet = maxpool2d(convnet)

convnet = tf.nn.relu(conv2d(convnet, weights['Wconv4']) + biases['bconv4'])
convnet = maxpool2d(convnet)

convnet = tf.nn.relu(conv2d(convnet, weights['Wconv5']) + biases['bconv5'])
convnet = maxpool2d(convnet)

convnet = tf.reshape(convnet, [-1, 2*2*32])
convnet = tf.nn.relu(tf.matmul(convnet, weights['Wfc']) + biases['bfc'])
convnet = tf.nn.dropout(convnet, keepRate)

convnet = tf.matmul(convnet, weights['Wout']) + biases['bout']

In [32]:
prediction = convnet
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=prediction))
optimizer = tf.train.AdadeltaOptimizer(LR).minimize(loss)
correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

print('Total Trainable Parameters: ', np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()]))

Total Trainable Parameters:  13078952


In [33]:
# trainX = np.array([i for i in train_data[:, 0]]).reshape([-1, 50, 50, 3])
# trainY = np.array([[i[0], i[1]] for i in train_data[:, 1]])
# t_trainX = trainX[:500]
# t_trainY = trainY[:500]

trainX = train_data_imgs
trainY = train_data_lbls
t_trainX = trainX[:500]
t_trainY = trainY[:500]

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        currentBatch = 0
        while currentBatch < len(train_data_imgs):
            batchX = trainX[currentBatch:currentBatch+batchSize]
            batchY = trainY[currentBatch:currentBatch+batchSize]
            currentBatch += batchSize
            sess.run([optimizer], feed_dict={x: batchX, y: batchY})
            
        l, a = sess.run([loss, accuracy], feed_dict={x: t_trainX, y: t_trainY})
        print('Epoch: ', epoch+1, '\nLoss: ', l, '\nAccuracy: ', a)
        print('\n'*2)

Epoch:  1 
Loss:  25562681000.0 
Accuracy:  0.512



Epoch:  2 
Loss:  14415293000.0 
Accuracy:  0.564



Epoch:  3 
Loss:  11634263000.0 
Accuracy:  0.512



Epoch:  4 
Loss:  8572043300.0 
Accuracy:  0.546



Epoch:  5 
Loss:  7219994000.0 
Accuracy:  0.53



Epoch:  6 
Loss:  5755586000.0 
Accuracy:  0.548



Epoch:  7 
Loss:  4765586400.0 
Accuracy:  0.586



Epoch:  8 
Loss:  4280105700.0 
Accuracy:  0.54



Epoch:  9 
Loss:  3437640000.0 
Accuracy:  0.57



Epoch:  10 
Loss:  3104293000.0 
Accuracy:  0.542



Epoch:  11 
Loss:  2804925700.0 
Accuracy:  0.548



Epoch:  12 
Loss:  2103194600.0 
Accuracy:  0.566



Epoch:  13 
Loss:  1856204900.0 
Accuracy:  0.538



Epoch:  14 
Loss:  1290750000.0 
Accuracy:  0.568



Epoch:  15 
Loss:  1106596500.0 
Accuracy:  0.566



Epoch:  16 
Loss:  675527550.0 
Accuracy:  0.586



Epoch:  17 
Loss:  641782900.0 
Accuracy:  0.522



Epoch:  18 
Loss:  373134180.0 
Accuracy:  0.542



Epoch:  19 
Loss:  309853700.0 
Accuracy:  0.526



Epoch: